In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
!ln -s "/content/drive/My Drive/Estudos/unifor/Oitavo/Sistemas inteligentes/perceptron" /content/dados

ln: failed to create symbolic link '/content/dados/perceptron': File exists


# Explorando

In [42]:
import pandas as pd

In [43]:
treino = pd.read_csv('dados/dataset-treinamento.csv')

In [44]:
treino.head()

,x1,x2,x3,x4,d
0,0.4329,-1.3719,0.7022,-0.8535,1.0
1,0.3024,0.2286,0.8630,2.7909,-1.0
2,0.1349,-0.6445,1.0530,0.5687,-1.0
3,0.3374,-1.7163,0.3670,-0.6283,-1.0
4,1.1434,-0.0485,0.6637,1.2606,1.0


In [45]:
treino.d = treino.d.astype('int')

# Perceptron

In [46]:
import numpy as np
class Perceptron:
    def __init__(self,input_values,output_values,learn_rate,activation_function):
        self.input_values=input_values
        self.output_values=output_values
        self.learn_rate=learn_rate
        self.activation_function=activation_function
        self.W= np.random.rand(input_values.shape[1])
        self.theta = np.random.rand(1)[0]
        
        self.Wi=np.concatenate(([self.theta],self.W))
    def evaluate(self,x):
        u= np.dot(x,self.W)-self.theta
        return self.activation_function(u)
    def train (self,n=-1):
        epochs = 1
        error = True
        # print(f'inicial W: {self.W}')
        # print()
        while error and (n>0 or n==-1):
            error = False
            # print('epoca',epochs)
            
            for x,d in zip(self.input_values,self.output_values):
                u= np.dot(x,self.W)-self.theta
                y= self.activation_function(u)
                
                # print(f'input: {x}, ouput: {y}, expected: {d}')

                if y!=d:
                    # print('recalculando W')
                    # print(f'atual W: {self.W}')

                    self.W=self.W+self.learn_rate*(d-y)*x
                    self.theta=self.theta+self.learn_rate*(d-y)*-1
                    error = True
                    
                    # print(f'novo W: {self.W}')
                    break
            if n >0:
                n-=1
            epochs+=1
            wf=np.concatenate(([self.theta],self.W))
        return self.Wi,wf,epochs
        print(f'final W: {self.W}')
                

In [47]:
def binary_step(x):
    return 1 if x>=0 else 0
def sign_function(x):
    return 1 if x>=0 else -1

# Treinamento

In [48]:
Y = treino['d'].values
X = treino[['x1','x2','x3','x4']].values

# Teste

In [49]:
teste = pd.read_csv('dados/dataset-teste.csv')

In [50]:
teste.head()

,x1,x2,x3,x4
0,0.9694,0.6909,0.4334,3.4965
1,0.5427,1.3832,0.6390,4.0352
2,0.6081,-0.9196,0.5925,0.1016
3,-0.1618,0.4694,0.2030,3.0117
4,0.1870,-0.2578,0.6124,1.7749


In [51]:
Resultado=treino.copy()
Treinamentos = pd.DataFrame()
for i in range(5):

    activation_function = sign_function
    perceptron = Perceptron(X,Y,0.01, activation_function)
    wi,wf,ep = perceptron.train(200000)
    wi = [round(x,4) for x in wi] 
    wf = [round(x,4) for x in wf]
    w = np.concatenate(([wi],[wf],[[ep]]),axis=1)
    
    Treino = pd.DataFrame(data=w,columns=['Wi0','Wi1','Wi2','Wi3','Wi4','Wf0','Wf1','Wf2','Wf3','Wf4','N_Epocas'],index=[f'T{i+1}'])
    Treinamentos= pd.concat([Treinamentos,Treino])

    resultado_parcial=pd.DataFrame(columns=[f'T{i+1}'])
    respostas=[]
    for j in treino.values:
        respostas.append(perceptron.evaluate(j[:-1]))

    resultado_parcial=pd.DataFrame(data=respostas,columns=[f'T{i+1}'])
    Resultado= pd.concat([Resultado,resultado_parcial],axis=1)
    

In [52]:
print(Treinamentos)

       Wi0     Wi1     Wi2     Wi3  ...     Wf2     Wf3     Wf4  N_Epocas
T1  0.0401  0.1617  0.1933  0.6669  ...  0.5022  0.0085 -0.3522    1538.0
T2  0.6953  0.3325  0.8934  0.5371  ...  0.5713 -0.0032 -0.4110    1514.0
T3  0.2527  0.0855  0.5436  0.3661  ...  0.5352  0.0007 -0.3743    1468.0
T4  0.1777  0.5731  0.2364  0.1178  ...  0.5940 -0.0039 -0.4221    1629.0
T5  0.7917  0.1030  0.8783  0.1665  ...  0.5583  0.0016 -0.3916    1680.0

[5 rows x 11 columns]


In [53]:
print(Resultado)

        x1      x2      x3      x4  d  T1  T2  T3  T4  T5
0   0.4329 -1.3719  0.7022 -0.8535  1   1   1   1   1   1
1   0.3024  0.2286  0.8630  2.7909 -1  -1  -1  -1  -1  -1
2   0.1349 -0.6445  1.0530  0.5687 -1  -1  -1  -1  -1  -1
3   0.3374 -1.7163  0.3670 -0.6283 -1  -1  -1  -1  -1  -1
4   1.1434 -0.0485  0.6637  1.2606  1   1   1   1   1   1
5   1.3749 -0.5071  0.4464  1.3009  1   1   1   1   1   1
6   0.7221 -0.7587  0.7681 -0.5592  1   1   1   1   1   1
7   0.4403 -0.8072  0.5154 -0.3129  1   1   1   1   1   1
8  -0.5231  0.3548  0.2538  1.5776 -1  -1  -1  -1  -1  -1
9   0.3255 -2.0000  0.7112 -1.1209  1   1   1   1   1   1
10  0.5824  1.3915 -0.2291  4.1735 -1  -1  -1  -1  -1  -1
11  0.1340  0.6081  0.4450  3.2230 -1  -1  -1  -1  -1  -1
12  0.1480 -0.2988  0.4778  0.8649  1   1   1   1   1   1
13  0.7359  0.1869 -0.0872  2.3584  1   1   1   1   1   1
14  0.7115 -1.1469  0.3394  0.9573 -1  -1  -1  -1  -1  -1
15  0.8251 -1.2840  0.8452  1.2382 -1  -1  -1  -1  -1  -1
16  0.1569  0.